In [1]:
import pandas as pd
import numpy as np

In [2]:
# read data

data = pd.read_csv('data.csv')
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# split into train and test dataset

ratio = 0.67
n_train = int(len(data) * ratio)
n_test = len(data) - n_train
indexes = np.zeros(data.shape[0])
indexes[np.random.choice(len(data), n_train, replace=False)] = 1

X = np.array(data[indexes == 1].values)[:,:-1]
y = np.array(data[indexes == 1].values)[:,-1]
test_x = np.array(data[indexes == 0].values)[:,:-1]
test_y = np.array(data[indexes == 0].values)[:,-1]

In [4]:
X = np.array(X, dtype=np.float32)
means, stds = X.mean(axis=0), X.std(axis=0)
X = (X - means) / stds

In [5]:
# each one vs all
y_setosa = [1 if x == 'Iris-setosa' else 0 for x in y]
y_versicolor = [1 if x == 'Iris-versicolor' else 0 for x in y]
y_virginica = [1 if x == 'Iris-virginica' else 0 for x in y]

In [6]:
X = np.hstack((np.ones((X.shape[0], 1)), X))

In [7]:
def mserror(y, y_pred):
    # count mean square error of prediction"
    return sum((y-y_pred)**2)

In [8]:
def hypothesis(X, w):
    # count hypothesis (sigmoid function)
    return 1 / (1 + np.exp (-(X.dot(w))))

In [9]:
def stochastic_gradient_step(X, y, w, train_ind, eta=0.01):
    # count gradient for current iteration
    hyp = hypothesis(X[train_ind], w)
    grads = []
    for i in range(len(X[train_ind])):
        grad_i = X[train_ind][i] * (hyp - y[train_ind])
        grads.append(grad_i)
    return  w - 2 * eta / len(X) * np.array(grads)

In [10]:
def stochastic_gradient_descent(X, y, w_init, eta=1e-2, max_iter=1e4,
                                min_weight_dist=1e-8, seed=42, verbose=False):
    # use stochastic gradient descent in order to find the most exact weights
    weight_dist = np.inf
    w = w_init
    errors = []
    iter_num = 0
    np.random.seed(seed)
        
    while weight_dist > min_weight_dist and iter_num < max_iter: 
        
        random_ind = np.random.randint(X.shape[0])        
        
        w_moved = stochastic_gradient_step(X, y, w, random_ind, eta)
        weight_dist = np.sqrt(np.sum((w-w_moved)**2))                                      
        w = w_moved
        
        y_pred = hypothesis(X, w)
        errors.append(mserror(y, y_pred))
        
        if verbose:
            print(w)
            
        iter_num += 1
        
    return w, errors

In [11]:
%%time
w_init = np.full(X.shape[1], 0)
setosa_weights, setosa_errors = stochastic_gradient_descent(X, y_setosa, w_init, min_weight_dist = 1e-10, max_iter=1e4, verbose=False)
print(setosa_weights)

[-0.29877384 -0.35430687  0.33024455 -0.49134003 -0.47252876]
CPU times: user 611 ms, sys: 3.8 ms, total: 615 ms
Wall time: 614 ms


In [12]:
%%time
w_init = np.full(X.shape[1], 0)
versicolor_weights, versicolor_errors = stochastic_gradient_descent(X, y_versicolor, w_init, min_weight_dist = 1e-10, max_iter=1e4, verbose=False)
print(versicolor_weights)

[-0.20786343  0.05488427 -0.28526914  0.11225816  0.04626529]
CPU times: user 639 ms, sys: 1.89 ms, total: 641 ms
Wall time: 639 ms


In [13]:
%%time
w_init = np.full(X.shape[1], 0)
virginica_weights, virginica_errors = stochastic_gradient_descent(X, y_virginica, w_init,  min_weight_dist = 1e-10, max_iter=1e4, verbose=False)
print(virginica_weights)

[-0.29962678  0.28266338 -0.03914002  0.37255245  0.41913918]
CPU times: user 633 ms, sys: 2.84 ms, total: 636 ms
Wall time: 635 ms


In [14]:
def classify(X, y, setosa_weights, versicolor_weights, virginica_weights, verbose = False):
    y_pred = []
    for row in X:
        x = row
        h_setosa = hypothesis(x, setosa_weights)
        h_versicolor = hypothesis(x, versicolor_weights)
        h_virginica = hypothesis(x, virginica_weights)
        if max(h_setosa, h_versicolor, h_virginica) == h_setosa:
            y_pred.append('Iris-setosa')
        elif max(h_setosa, h_versicolor, h_virginica) == h_versicolor:
            y_pred.append('Iris-versicolor')
        elif max(h_setosa, h_versicolor, h_virginica) == h_virginica:
            y_pred.append('Iris-virginica')
        if verbose:
            print (h_setosa, h_versicolor, h_virginica)
    y = pd.DataFrame(y)
    y.columns = ['class']
    y['predicted'] = y_pred
    return y            

In [15]:
# means, stds from sample
test_x = (test_x - means) / stds

In [16]:
test_x = np.hstack((np.ones((test_x.shape[0], 1)), test_x))

In [17]:
def accuracy(test_y):
    return sum(test_y['class'] == test_y['predicted']) / len (test_y)

In [18]:
test_y = classify(test_x, test_y, setosa_weights, versicolor_weights, virginica_weights)
test_y

,class,predicted
0,Iris-setosa,Iris-setosa
1,Iris-setosa,Iris-setosa
2,Iris-setosa,Iris-setosa
3,Iris-setosa,Iris-setosa
4,Iris-setosa,Iris-setosa
5,Iris-setosa,Iris-setosa
6,Iris-setosa,Iris-setosa
7,Iris-setosa,Iris-setosa
8,Iris-setosa,Iris-setosa
9,Iris-setosa,Iris-setosa


In [19]:
print ('Accuracy =', accuracy(test_y))

Accuracy = 0.92


In [20]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [21]:
# Create linear regression object
regr = linear_model.LogisticRegression()

# Train the model using the training sets
regr.fit(X, y)

# Make predictions using the testing set
sklearn_test_y = regr.predict(test_x)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error

Coefficients: 
 [[-0.97104117 -0.81333323  1.07803112 -1.54900411 -1.42861203]
 [-0.3127344   0.19521639 -0.8724833   0.57048635 -0.60068717]
 [-1.42122197  0.04076386 -0.41047022  2.02419152  2.41634118]]


In [22]:
test_y['sklearn'] = sklearn_test_y
test_y

,class,predicted,sklearn
0,Iris-setosa,Iris-setosa,Iris-setosa
1,Iris-setosa,Iris-setosa,Iris-setosa
2,Iris-setosa,Iris-setosa,Iris-setosa
3,Iris-setosa,Iris-setosa,Iris-setosa
4,Iris-setosa,Iris-setosa,Iris-setosa
5,Iris-setosa,Iris-setosa,Iris-setosa
6,Iris-setosa,Iris-setosa,Iris-setosa
7,Iris-setosa,Iris-setosa,Iris-setosa
8,Iris-setosa,Iris-setosa,Iris-setosa
9,Iris-setosa,Iris-setosa,Iris-setosa
